In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# data loader

class dataLoader:
    """
    Load data 
    """
    def __init__(self, train_filename, test_filename, normalize = ['Age', 'Fare']):
        
        self.train = pd.read_csv(train_filename)
        self.test  = pd.read_csv(test_filename)
        
        for cts_variable in normalize:
            self.normalize(cts_variable)
        
        #self.remove_na()
        #self.train_label = self.train.pop('Survived')
    
    def remove_na(self):
        """ Remove empty or invalid elements"""
        self.train.fillna(0, inplace = True)
        self.test.fillna(0, inplace = True)
    
    def normalize(self, column_name):
        """ Normalize continuous data to zero mean and standard deviation of 1"""
        
        mu = self.train[column_name].mean()
        std = self.train[column_name].std()
        
        self.train[column_name] = (self.train[column_name] - mu) / std
        self.test[column_name] = (self.test[column_name] - mu) / std
    
    def summary(self):
        """ Output summary of data and first few rows"""
        print('Training set:')
        #print(self.train.head())
        print(self.train.describe())
        
        print('Testing set:')
        #print(self.test.head())
        print(self.test.describe())
        
    
raw_data = dataLoader('/kaggle/input/titanic/train.csv', '/kaggle/input/titanic/test.csv')
raw_data.train
raw_data.test

test_ids = raw_data.test[['PassengerId']]
test_ids

,PassengerId
0,892
1,893
2,894
3,895
4,896
...,...
413,1305
414,1306
415,1307
416,1308


In [3]:
# remove features that are not useful for now
raw_data.train.drop(columns = ['PassengerId', 'Ticket', 'Name'], inplace=True)
raw_data.test.drop(columns = ['PassengerId', 'Ticket', 'Name'], inplace=True)

# take the first element of the cabin feature
raw_data.train['Cabin'] = raw_data.train['Cabin'].str[0]
raw_data.test['Cabin'] = raw_data.test['Cabin'].str[0]

# encode class features using one hot encoding - try combining test to get same number of splits
raw_data.train = pd.get_dummies(raw_data.train, columns = ['Sex', 'Embarked', 'Cabin'], drop_first = True)
raw_data.test = pd.get_dummies(raw_data.test, columns = ['Sex', 'Embarked', 'Cabin'], drop_first = True)

# remove nan values from age, ...
raw_data.remove_na()
train_label = raw_data.train.pop('Survived')

# ensure train and test sets have same number of columns
missing_cols = set( raw_data.train.columns ) - set( raw_data.test.columns )
# Add a missing column in test set with default value equal to 0
for c in missing_cols:
    raw_data.test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
raw_data.test = raw_data.test[raw_data.train.columns]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(raw_data.train, train_label, test_size=0.3, random_state=0)

# create model (RF, XGBoost, ...)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model = model.fit(X_train, y_train)
test_acc = model.score(X_test, y_test)
test_acc

0.8022388059701493

In [5]:
from sklearn import svm
model = svm.SVC(kernel = 'rbf', degree = 2, verbose = True)
model = model.fit(X_train, y_train)
test_acc = model.score(X_test, y_test)
test_acc

[LibSVM]

0.8097014925373134

In [6]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators =80, max_depth = 9, random_state = 0)
model = model.fit(X_train, y_train)
test_acc = model.score(X_test, y_test)
test_acc

0.8171641791044776

In [7]:
# save predictions
test_pred = model.predict(raw_data.test)
#np.shape(test_pred)

submission = test_ids
submission["Survived"] = test_pred

submission.to_csv('/kaggle/working/submission.csv', index = False)
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [8]:
# ensemble models (Decision tree)

In [9]:
# hyperparameter tuning (GA?)

In [10]:
# model evaluation (confusion matrics, accuracy)